In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [3]:
import getpass
import os


def _set_env(var: str):
    os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

In [7]:
from langchain_core.callbacks import Callbacks
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI


@tool
async def get_items(
    place: str,
) -> str:
    """Use this tool to look up which items are in the given place"""
    # Attention when using async, you should be invoking the LLM using ainvoke!
    # If you fail to do so, streaming will not WORK.
    return await llm.ainvoke(
        [
            {
                "role": "user",
                "content": f"Can you tell me what kind of items i might find in the following place: '{place}'. "
                "List at least 3 such items separating them by a comma. And include a brief description of each item..",
            }
        ],
    )


llm = ChatOpenAI(model_name="gpt-4o")
tools = [get_items]
agent = create_react_agent(llm, tools=tools)

In [ ]:
final_message = ""
async for msg, metadata in agent.astream(
    {"messages": [("human", "what items are on the pantry shelf?")]}, stream_mode="messages"
):
    # Stream all messages from the tool node
    if (
        msg.content
        and not isinstance(msg, HumanMessage)
        and metadata["langgraph_node"] == "tools"
        and not msg.name
    ):
        print(msg.content, end="|", flush=True)
    # Final message should come from our agent
    if msg.content and metadata["langgraph_node"] == "agent":
        final_message += msg.content

In [12]:
from langchain_core.messages import HumanMessage

async for event in agent.astream_events(
    {"messages": [{"role": "user", "content": "what's in the bedroom."}]}, version="v2"
):
    if (
        event["event"] == "on_chat_model_stream"
        and event["metadata"].get("langgraph_node") == "tools"
    ):
        print(event["data"]["chunk"].content, end="|", flush=True)

|In| a| bedroom|,| you| might| typically| find| the| following| items|:

|1|.| **|Bed|**|:| The| primary| piece| of| furniture| for| sleeping|,| usually| consisting| of| a| mattress| on| a| frame|,| and| often| accompanied| by| bedding| such| as| sheets|,| blankets|,| and| pillows|.

|2|.| **|Ward|robe|**|:| A| tall|,| standalone| cabinet| used| for| storing| clothes|,| shoes|,| and| accessories|.| It| often| includes| shelves|,| drawers|,| and| hanging| space| to| help| organize| clothing|.

|3|.| **|Night|stand|**|:| A| small| table| or| cabinet| placed| beside| the| bed|,| used| for| holding| items| that| might| be| needed| during| the| night|,| such| as| a| lamp|,| alarm| clock|,| or| personal| belongings| like| a| phone| or| books|.||